In [77]:
import pandas as pd
import folium
from folium.plugins import HeatMap


In [78]:
data = pd.read_csv('/Users/haritshah/Desktop/Dataset/AEGISDataset.csv')
data.head()


,lat,lon,flood_heig,elevation,precipitat
0,14.640394,121.055708,0,54.553295,9.0
1,14.698299,121.002132,0,21.856272,10.0
2,14.698858,121.100261,0,69.322807,16.0
3,14.571310,120.983334,0,10.987241,8.0
4,14.762232,121.075735,0,87.889847,18.0


In [79]:
numeric_data = data[['lat', 'lon', 'flood_heig', 'elevation', 'precipitat']]
data[numeric_data.columns] = numeric_data.apply(pd.to_numeric, errors='coerce')
data_clean = data.dropna()
print("Nan after cleaning:")
print(data_clean.isnull().sum())

Nan after cleaning:
lat           0
lon           0
flood_heig    0
elevation     0
precipitat    0
dtype: int64


In [80]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_clean['flood_heig_normalized'] = scaler.fit_transform(data_clean[['flood_heig']])
print(data_clean[['flood_heig', 'flood_heig_normalized']].head())
# Min-Max normalization for elevation and precipitat
data_clean['elevation_normalized'] = (data_clean['elevation'] - data_clean['elevation'].min()) / (data_clean['elevation'].max() - data_clean['elevation'].min())
data_clean['precipitat_normalized'] = (data_clean['precipitat'] - data_clean['precipitat'].min()) / (data_clean['precipitat'].max() - data_clean['precipitat'].min())
print(data_clean[['elevation', 'elevation_normalized', 'precipitat', 'precipitat_normalized']].head())

# Alternatively, we can use MinMaxScaler for normalization
# data_clean[['elevation_normalized', 'precipitat_normalized']] = scaler.fit_transform(data_clean[['elevation', 'precipitat']])

   flood_heig  flood_heig_normalized
0           0                    0.0
1           0                    0.0
2           0                    0.0
3           0                    0.0
4           0                    0.0
   elevation  elevation_normalized  precipitat  precipitat_normalized
0  54.553295              0.594044         9.0               0.428571
1  21.856272              0.237998        10.0               0.476190
2  69.322807              0.754873        16.0               0.761905
3  10.987241              0.119643         8.0               0.380952
4  87.889847              0.957054        18.0               0.857143


In [81]:
data_clean['elevation_normalized'] = 1 - data_clean['elevation_normalized']

In [82]:
data_clean["risk_score"]  = (data_clean['flood_heig_normalized'] * 0.4 + data_clean['elevation_normalized'] * 0.3 + data_clean['precipitat_normalized']*0.3)

In [83]:
print("\Risk Score for each location:")
for index, row in data_clean.iterrows():
    print(f"Latitude: {row['lat']}, Longitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}")

\Risk Score for each location:
Latitude: 14.64039374, Longitude: 121.0557085, Flood Height: 0.0, Risk Score: 0.2503582609426578
Latitude: 14.69829935, Longitude: 121.0021322, Flood Height: 0.0, Risk Score: 0.3714576900771931
Latitude: 14.69885841, Longitude: 121.1002614, Flood Height: 0.0, Risk Score: 0.30210963466319884
Latitude: 14.57131006, Longitude: 120.9833345, Flood Height: 0.0, Risk Score: 0.3783929056101954
Latitude: 14.7622323, Longitude: 121.0757349, Flood Height: 0.0, Risk Score: 0.2700267817151137
Latitude: 14.6011784, Longitude: 121.0885987, Flood Height: 0.0, Risk Score: 0.4098931394636641
Latitude: 14.70803089, Longitude: 121.052674, Flood Height: 0.0, Risk Score: 0.3244220322521432
Latitude: 14.70920612, Longitude: 121.0502922, Flood Height: 0.0, Risk Score: 0.3445020220605377
Latitude: 14.70962531, Longitude: 121.0470059, Flood Height: 0.0, Risk Score: 0.3303591941495462
Latitude: 14.66933825, Longitude: 121.1328981, Flood Height: 0.0, Risk Score: 0.38514977754177726


In [84]:
# Define a function to assign colors based on risk score
def get_color(risk_score):
    if risk_score < 0.2:
        return 'green'
    elif 0.2 <= risk_score < 0.4:
        return 'yellow'
    elif 0.4 <= risk_score < 0.6:
        return 'orange'
    elif 0.6 <= risk_score < 0.8:
        return 'red'
    else:
        return 'darkred'
    

# # Add markers to the map
# for index, row in data_clean.iterrows():
#     folium.CircleMarker(
#         location=[row['lat'], row['lon']],
#         radius=5,
#         color=get_color(row['risk_score']),
#         fill=True,
#         fill_color=get_color(row['risk_score']),
#         fill_opacity=0.7
#     ).add_to(m)

# # Add heat map
# heat_data = [[row['lat'], row['lon'], row['risk_score']] for index, row in data_clean.iterrows()]
# HeatMap(heat_data).add_to(m)

# # Display the map
# m

In [85]:
import branca

# Add risk color to the data
data_clean['risk_color'] = data_clean['risk_score'].apply(get_color)

# Define the coordinates for Manila
manila_coords = [14.5995, 120.9842]

# Create a folium map centered on Manila
map_flood_risk = folium.Map(location=manila_coords, zoom_start=12)

# Add markers to the map
for index, row in data_clean.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color=row['risk_color'],
        fill=True,
        fill_color=row['risk_color'],
        fill_opacity=0.7
    ).add_to(map_flood_risk)

# Create a legend
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 150px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity: 0.8;">
    &nbsp;<b>Flood Risk Score</b><br>
    &nbsp;<i class="fa fa-circle" style="color:green"></i>&nbsp;Low (0.0 - 0.2)<br>
    &nbsp;<i class="fa fa-circle" style="color:yellow"></i>&nbsp;Moderate (0.2 - 0.4)<br>
    &nbsp;<i class="fa fa-circle" style="color:orange"></i>&nbsp;High (0.4 - 0.6)<br>
    &nbsp;<i class="fa fa-circle" style="color:red"></i>&nbsp;Very High (0.6 - 0.8)<br>
    &nbsp;<i class="fa fa-circle" style="color:darkred"></i>&nbsp;Severe (0.8 - 1.0)
</div>
'''

map_flood_risk.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
map_flood_risk.save('/Users/haritshah/Desktop/Dataset/flood_risk.html')

# Display the map
map_flood_risk
